In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


**To transfer data from dropbox to surfsara**
- select the folders of interest within the folders of dropbox

In [4]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Rachael_set/"

if not os.path.exists(directory_targ):
    os.makedirs(directory_targ)
    
### IS THIS RACHAEL'S DATASET OR SOMEONE WHO HAS VIDEOINFO.TXT IN THEIR FOLDER???

all_folders_drop, excel_drop, txt_drop = get_dropbox_video_folders("/DATA/FLUORESCENCE/DATA_NileRed/", True)

# folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"] == "452"]
all_folders_drop = pd.read_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

clear_output(wait=False)

all_folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230123_Plate530_001,530,20230123,DATA/FLUORESCENCE/DATA_NileRed/20230123_Plate5...,1
1,20230123_Plate530_002,530,20230123,DATA/FLUORESCENCE/DATA_NileRed/20230123_Plate5...,2
2,20230123_Plate530_003,530,20230123,DATA/FLUORESCENCE/DATA_NileRed/20230123_Plate5...,3
3,20230123_Plate530_004,530,20230123,DATA/FLUORESCENCE/DATA_NileRed/20230123_Plate5...,4
4,20230123_Plate530_005,530,20230123,DATA/FLUORESCENCE/DATA_NileRed/20230123_Plate5...,5
...,...,...,...,...,...
800,20230201_Plate558_37,558,20230201,DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate5...,37
801,20230201_Plate558_38,558,20230201,DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate5...,38
802,20230201_Plate558_39,558,20230201,DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate5...,39
803,20230201_Plate558_40,558,20230201,DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate5...,40


In [13]:
# directory_targ = "/gpfs/scratch1/shared/amftrackflow/Rachael_dataset/"
# all_folders_drop.to_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

# save_dropbox_state("/DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate552/", is_video=True)

folders_drop = all_folders_drop
folders_drop = folders_drop.loc[folders_drop["Plate number"] == 532]
folders_drop = folders_drop.loc[folders_drop["video"] == 15]

In [14]:
folders_drop = folders_drop.sort_values(by=['folder'], ignore_index=True)
# folders_drop["folder"] = [os.path.relpath(entry, "DATA/FLUORESCENCE/DATA_NileRed/") for entry in folders_drop["tot_path_drop"]]
folders_drop["video"]  = [entry.split('_')[-1] for entry in folders_drop["tot_path_drop"]]
folders_drop["folder"] = [f"{entry.split('/')[-3]}/{entry.split('_')[-1]}/Img" for entry in folders_drop["tot_path_drop"]]

# folders_drop = folders_drop.loc[folders_drop["video"] == "005"]

In [15]:
# FINAL CHECK

print(directory_targ)
print(folders_drop["tot_path_drop"].iloc[0])
print(folders_drop["Date Imaged"].unique())
folders_drop

/gpfs/scratch1/shared/amftrackflow/Rachael_set/
DATA/FLUORESCENCE/DATA_NileRed/20230125_Plate532/DATA/20230125_Plate532_15
[20230125]


,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230125_Plate532/15/Img,532,20230125,DATA/FLUORESCENCE/DATA_NileRed/20230125_Plate5...,15


In [16]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    1,
    "10:00:00",
    "transfer_test"
)


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2945616
Submitted batch job 2945617


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


In [73]:
# print(excel_drop)
for xl_adress in excel_drop:
    file_name = f"{xl_adress.split('/')[-2]}/{xl_adress.split('/')[-1]}"
    if file_name.split(".")[-2][-4:] == 'data':
        continue
    print(file_name.split(".")[-2][-4:])

    download(xl_adress, directory_targ+file_name)

e462
2023-06-16 23:29:45,748-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 23:29:45,909-[INFO]- dropbox:474 -> Request to files/download
e530
2023-06-16 23:29:46,436-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 23:29:46,622-[INFO]- dropbox:474 -> Request to files/download
e462
2023-06-16 23:29:47,015-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 23:29:47,163-[INFO]- dropbox:474 -> Request to files/download
e545
2023-06-16 23:29:47,528-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 23:29:47,698-[INFO]- dropbox:474 -> Request to files/download
e510
2023-06-16 23:29:48,399-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 23:29:48,549-[INFO]- dropbox:474 -> Request to files/download
e452
2023-06-16 23:29:48,969-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 23:29:49,133-[INFO]- dropbox:474 -> Request to files/download
e452
2023-06-16 23:29:49,676-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 23:29:49,826